In [14]:
import numpy as np
import os
from PIL import Image
from time import time

# import tensorflow as tf
from tensorflow.keras.layers import AveragePooling2D

### Load Your Pictures into the Images Folder

The images folder is where the code will look for pictures to generate the ASCII versions of. The final output will be written to the output folder. Input images will be converted to JPEG files but it is preferable to upload native JPEG images first.

Once the ASCII "images" are written to a txt file, you can download them and view them in a .txt file editor like Text Edit. If you have chosen a small kernel size, you will need to zoom out a fair bit to view the entire image.

If the code is run multiple times, the code will not remake the directories.

In [15]:
try:
    os.mkdir("images")
except FileExistsError:
    print("Images directory already exists")

try:
    os.mkdir("output")
except FileExistsError:
    print("Output directory already exists")

input_path = "images"
output_path = "output"

output directory already exists


The input files will be converted to .jpeg files

In [ ]:
for file in os.listdir(input_path):
    fname, ftype = file.split(".")
    if fname != "" and ftype != "jpeg":
        file = os.rename(f"{input_path}/{file}", f"{input_path}/{fname}.jpeg")

In [16]:
# ordered gray scale character list
gscale = "$@B%8&WM#*oahkbdpqwmZO0QLCJUYXzcvunxrjft/\|()1{}[]?-_+~<>i!lI;:,\"^`'. "

# convert grey scale to dict to speed up look up, invert it for negative images
gs_dict = {index: value for (index, value) in enumerate(gscale)}
gs_dict_inv = {index: value for (index, value) in enumerate(reversed(gscale))}


This function converts the image to grayscale by averaging the RGB channels of the raw image matrix.

In [18]:
def convert_to_gs(image_array):
    return np.mean(np.asarray(image_array), axis=2)


This timer decorator, written by GeeksforGeeks, measures the wall clock time to generate an image. It takes anywhere from 0.5 to 2 seconds depending on the size of the image and the kernel.

In [19]:
def timer(func):
    def wrap_func(*args, **kwargs):
        t1 = time()
        result = func(*args, **kwargs)
        t2 = time()
        print(f"Image generated in {(t2-t1):.4f}s")
        return result

    return wrap_func

This function converts the image to the final ASCII output, conditional on the size of the kernel. The kernel maps the number of pixels that correspond to each ASCII character. Large kernel sizes result in less detail, small kernel sizes, more detail. 

In [20]:
@timer
def ascii_conv(image, kernel_size, output, invert: bool, negative: bool):
    # take in the image and transform it into an array
    image_array = convert_to_gs(np.array(image))

    if invert:
        if image_array.shape[1] > image_array.shape[0]:
            image_array = image_array.T

    # reshape the image so it is a valid 4D tensor for pooling
    image_array = image_array.reshape(1, image_array.shape[0], image_array.shape[1], 1)

    # average a neighborhood of pixels to get the luminosity of each tile
    avg_pool_2d = AveragePooling2D(
        pool_size=(kernel_size, kernel_size), strides=None, padding="valid"
    )
    luminosity_array = np.asarray(avg_pool_2d(image_array))

    # map luminosity values to chars in grayscale list
    if negative:

        def char_map(i):
            return gs_dict_inv[int((i * 69) / 255)]

    else:

        def char_map(i):
            return gs_dict[int((i * 69) / 255)]

    mapping_function = np.vectorize(char_map)
    ascii_array = np.squeeze(mapping_function(luminosity_array))

    # return the ascii array as text
    return np.savetxt(f"{output_path}/{output}", ascii_array, fmt="%c")

The main function creates a list of input images to translate into ASCII output. Once the ASCII image is generated it will be written to the output folder, from there you can download it, open it with Text Edit on Mac or similar software on your OS and view the ASCII art. 

Note that when the kernel_size is quite small, images will take longer to render and may require a lot of zooming out to view the entire image.

A kernel size of 5 will create highly detailed images, whereas kernels around 15-25 will create classic ASCII style images.

In [21]:
def main(kernel_size=10, invert=None, negative=None):
    if kernel_size < 3:
        print(
            "WARNING: The minimum suggested kernel size is 3.\nSmaller kernel sizes will result in improperly rendered images.\nSuggested kernel sizes are between 5-25, depending on the size of the image and the desired level of detail."
        )
    # load and lazily store images
    images = []
    for file in os.listdir(input_path):
        fname, ftype = file.split(".")
        if fname != "" and ftype == "jpeg":
            images.append((Image.open(f"{input_path}/{file}"), fname))

    for i, image_tuple in enumerate(images):
        print("-" * 100)
        print(f"Image {i + 1} of {len(images)} loading:")
        print(f"{image_tuple[1]}, of size {image_tuple[0].size} rendering...")
        ascii_conv(
            image_tuple[0], kernel_size, f"{image_tuple[1]}_ascii.txt", invert, negative
        )
        print(f"ASCII art written to {output_path}/{image_tuple[1]}_ascii.txt")

In [22]:
main(5, invert=False, negative=True)
